In [1]:
import itertools

class Field(object):
    state = None
    parent = None
    children = []
    minimax_score = None
    
    def __init__(self, state):
        self.state = state
    
    def print_me(self):
        for line in self.state:
            print(*line, sep='')
    
    def is_move_of(self, side):
        cells = list(itertools.chain(*self.state))
        my_moves = len([c for c in cells if c == side])
        enemy_moves = len([c for c in cells if c not in (side, '.')])
        player = 'o' if my_moves == enemy_moves else 'x'
        return player == side

    def is_win_of(self, side):
        combinations = [
            [(0, 0), (0, 1), (0, 2)],
            [(1, 0), (1, 1), (1, 2)],
            [(2, 0), (2, 1), (2, 2)],
            [(0, 0), (1, 0), (2, 0)],
            [(0, 1), (1, 1), (2, 1)],
            [(0, 2), (1, 2), (2, 2)],
            [(0, 0), (1, 1), (2, 2)],
            [(2, 0), (1, 1), (0, 2)],
        ]
        for combo in combinations:
            if all(map(lambda p: self.state[p[0]][p[1]] == side, combo)):
                return True
        return False
    
    def is_tie(self):
        cells = list(itertools.chain(*self.state))
        return len([c for c in cells if c == '.']) == 0
    
    def get_next_move(self, best_for_side):
        best = None
        mx = -100000 if best_for_side == 'x' else 100000
        for child in self.children:
            if best_for_side == 'x':
                if child.minimax_score > mx:
                    mx, best = child.minimax_score, child
            else:
                if child.minimax_score < mx:
                    mx, best = child.minimax_score, child
        return best
    
    def get_minimax_score(self, level):
        if self.is_win_of('x'):
            self.minimax_score = 10
        elif self.is_win_of('o'):
            self.minimax_score = -10
        elif self.is_tie():
            self.minimax_score = 0
        elif level % 2 == 0:
            self.minimax_score = min([child.get_minimax_score(level + 1) for child in self.children])
        elif level % 2 == 1:
            self.minimax_score = max([child.get_minimax_score(level + 1) for child in self.children])
        return self.minimax_score

In [2]:
import copy

def generate_children(field):
    field.children = []
    if field.is_win_of('x') or field.is_win_of('o') or field.is_tie():
        return field.children
    else:
        for i in range(3):
            for j in range(3):
                if field.state[i][j] == '.':
                    field_copy = copy.deepcopy(field.state)
                    field_copy[i][j] = 'x' if field.is_move_of('o') else 'o'
                    field.children.append(Field(field_copy))
        return field.children


def update_minimax(field):    
    field.get_minimax_score(0)
    return field.minimax_score

In [3]:
# empty field
state0 = [['.', '.', '.'], ['.', '.', '.'], ['.', '.', '.']]

initial = Field(state0)
initial.print_me()

# generate a tree
states = []
states.append(initial)
i = 0
while i < len(states):
    states += generate_children(states[i])
    i += 1

print("Last field")
states[-1].print_me()

print('Total nodes in a tree:', len(states))

update_minimax(initial)
print("Root score: ", initial.minimax_score)
print("Last node score: ", states[-1].minimax_score)

...
...
...
Last field
xxo
oxo
xox
Total nodes in a tree: 549946
Root score:  0
Last node score:  10


In [5]:
field = initial
for side in ['x', 'o'] * 5:
    new_field = field.get_next_move(side)
    if new_field is not None:
        new_field.print_me()
        print("---")
        field = new_field
    else:
        break

x..
...
...
---
x.o
...
...
---
x.o
x..
...
---
x.o
x..
o..
---
x.o
xx.
o..
---
x.o
xx.
o.o
---
x.o
xxx
o.o
---
